# Stochastic Optimization

## 1.2 Simulated Annealing

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://supaerodatascience.github.io/stochastic/">https://supaerodatascience.github.io/stochastic/</a>

In [ ]:
from IPython.display import IFrame
IFrame("https://www.youtube.com/embed/GFVeYXJJndw", 560, 315)

Simulated annealing is a stochastic search algorithm which improves on the idea of random optimization by basing the search not only on the best found state, but also on random states. To determine which states to accept, it uses a term called *temperature*. The algorithm is inspired by the process of metal annealing, where a metal must cool in a slow process to reach a desired hardened configuration of minimal energy.

<img src="imgs/Simulated_Annealing.gif">

Because simulated annealing is based on the metal cooling metaphor, the literature surrounding it often refers to fitness values in the search space as energies, where the objective is to minimize the energy (cool the metal) slowly. Arriving at a local minimum of energy by cooling too rapidly creates non-optimal atomic configurations of the metal, as is seen in real metals.

<img src="imgs/simulated_annealing.png">
Delahaye, Daniel, Supatcha Chaimatanan, and Marcel Mongeau. "Simulated annealing: From basics to applications." Handbook of Metaheuristics. Springer, Cham, 2019. 1-35.

## Simulated Annealing

Let $f: \mathbb{R}^n → \mathbb{R}$ be the fitness or cost function which must be minimized. Let $x ∈ \mathbb{R}^n$ designate a position or candidate solution in the search-space.

    Initialize x randomly in ℝ
    for k in {0, kmax}
        x' = nearby(x)
        if f(x′) < f(x)
            x = x'
        else
            x = x' with probability P(f(x'), f(x), T)
    return x

`nearby(x)` is a function which finds a point near X. This is a flexible definition, and we can continue to use the definition of $x' = x + N(0,1)$.

The acceptance probability that $x = x'$ if $f(x') > f(x)$ is the difference between simulated annealing and random optimization seen in the last notebook. This probability allows the search to move away from local optima, accepting worse solutions in order to explore more of the search space. This probability depends on 3 terms: $f(x'), f(x),$ and $T$, the temperature. A common probability is:

$e^\frac{-(f(x')-f(x))}{T}$

The temperature $T$ is the factor which determines the rate of exploration of the search space as opposed to exploitation of a specific optima. Often, the schedule used is to start with $T=1$ and to decrease at each time step to $T=\frac{(k_{max}-k)}{k_{max}}$

Let's see some acceptance probabilities for different values of $f(x')$ and $T$ when $f(x)=1$.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import math

In [ ]:
fx = np.arange(1.0, 2.0, 0.1)
T = np.arange(1.0, 0.0, -0.1)
X, Y = np.meshgrid(fx, T)
proba = lambda fx, T :np.exp(-(fx - 1) / T)
Z = proba(X, Y)
fig = plt.figure(figsize=(8, 6))
cs = plt.contourf(X, Y, Z)
fig.colorbar(cs)
plt.show()

As we can see, for high temperature values (ie the beginning of the search), there is a high probability to accept solutions which are worse than our current best. However, as the search continues and the temperature decreases, the chance of accepting higher values decreases.

For a summary, here is the version of simulated annealing we can use for continuous optimization. Note that the choice of neighbor function, acceptance probability function, and temperature schedule are all parameters in simulated annealing and we could choose other functions suited to specific problems.

## Continuous Optimization Simulated Annealing

Let $f: \mathbb{R}^n → \mathbb{R}$ be the fitness or cost function which must be minimized. Let $x ∈ \mathbb{R}^n$ designate a position or candidate solution in the search-space.

    Initialize x randomly in ℝ
    for k in {0, kmax}
        x' = x + N(0, 1)
        T = (kmax - k) / kmax
        if (f(x′) < f(x)) or (exp(-(f(x')-f(x))/T) > rand())
            x = x'
    return x

<div class="alert alert-success">
    <h3>Exercise 3</h3>
Implement simulated annealing and compare it to random search. Does the acceptance probability help? Try tracking when it is used to see if more transitions are accepted early in the search. Compare random search and simulated annealing on the Himmelblau function. Are the conclusions different?
</div>

In [ ]:
# %load solutions/3_sa.py

<div class="alert alert-info">
    <h3>Discussion</h3>
At the beginning of class, we discussed the travelling salesman problem. This is a discrete optimization problem where a position in the search space represents the list of cities in the order that they will be visited. What would be a good `neighbor` function for this problem?
</div>